# Superdense coding



In [64]:
import pennylane as qml

dev = qml.device("default.qubit", wires=2, shots=10)

In [104]:
def bell_pair(wires=None):
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    
def decode(wires):
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.Hadamard(wires=wires[0])

In [115]:
@qml.qnode(dev)
def superdense_coding(bits):
    bell_pair(dev.wires)
    
    a, b = str(bits)
    if a == '1':
        qml.PauliZ(wires=0)
        
    if b == '1':
        qml.PauliX(wires=0)
        
    decode(dev.wires)
    
    return qml.sample()

In [116]:
superdense_coding('00')

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]], requires_grad=True)